In [1]:
import numpy as np
import csv
import os # Для проверки существования файла

from Polyhedra import *
from Distances import *
from Algorithm import *
from LLL import *

grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

grid1 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype = float)

grid2 = np.array([
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)

grid3 = np.array([
    [1, 1, 0, 0],
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1]
], dtype = float)

grid4 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)

grid5 = np.array([
    [2 ** 0.5, 0., 0., 0.],
    [-1/(2 ** 0.5), 6**0.5/3, 0., 0.],
    [0., -(2/3)**0.5, 2/((3**0.5)*10), 0.],
    [0., 0., -(3**0.5)/2, (5**0.5)/20]
], dtype = float)

# --- Имя файла для результатов ---
RESULTS_FILE_PATH = "optimal_results.csv"

# --- Заголовки для CSV файла ---
CSV_HEADERS = [
    "Grid_ID",
    "Grid_Matrix",
    "Determinant",
    "Mat_Matrix",
    "S_Point",
    #"Min_Center_Point",
    "Distance",
    # "Sub_Grid_LLL" # Раскомментируйте, если нужно
]
def format_matrix_for_csv(matrix):
    """Преобразует матрицу NumPy в строку для CSV."""
    if matrix is None:
        return ""
    # Каждую строку матрицы в строку, затем объединяем строки через точку с запятой (или другой разделитель)
    return ";".join([" ".join(map(str, row)) for row in matrix])

def format_point_for_csv(point):
    """Преобразует точку NumPy в строку для CSV."""
    if point is None:
        return ""
    return " ".join(map(str, point))





In [2]:
grid[0][0] = 2.15

print(grid)

#grid_int = (grid).astype(int)
#basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
#rows, cols = basis.nrows, basis.ncols
#grid = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

grid = LLL(grid, 0.75)

vor4 = VoronoiPolyhedra(grid)
vor4.build()

det_dist, det_center, det_mat = find_optimal(range(49, 50), 2, grid, vor4, vor4.max_len)



print(det_dist, det_center, det_mat)

[[2.15 0.   0.   0.  ]
 [1.   1.   0.   0.  ]
 [1.   0.   1.   0.  ]
 [1.   0.   0.   1.  ]]
суммарное расстояние = 67.03214342884613 индекс центрального региона = 329
Все точки используются в триангуляции.
Выпуклость триангуляции сохранена: True
                                                          
---------------------------------
det: 49
                                                          
▶ diag factors: 1 7 7    iters: 16807
 1.003964720365323 [-1.85 -1.   -3.    0.  ] [[1. 0. 0. 2.]
 [0. 1. 3. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.003964720365323 [-1.85 -1.   -3.    0.  ] [[1. 0. 1. 4.]
 [0. 1. 3. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0119555688976718 [0.7 0.  2.  3. ] [[1. 0. 1. 2.]
 [0. 1. 5. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0066471029519486 [-0.7 -2.  -3.   0. ] [[1. 0. 0. 4.]
 [0. 1. 5. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.003964720365324 [-1.85 -3.   -1.    0.  ] [[1. 0. 3. 0.]
 [0. 1. 0. 2.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.011955568897672 [-0.7 -3.   0.  

In [3]:
grid

array([[ 1.  ,  1.  ,  0.  ,  0.  ],
       [ 1.  ,  0.  ,  1.  ,  0.  ],
       [ 0.15, -1.  , -1.  ,  0.  ],
       [ 1.  ,  0.  ,  0.  ,  1.  ]])

In [4]:
# перебор матриц решетки (целочисленный вариант)

import itertools
from tqdm import tqdm
from math import comb

def normalize_rows(matrix):
    """
    Для каждой строки матрицы:
    - Находит первый ненулевой элемент
    - Если он отрицательный, умножает строку на -1
    """
    matrix = matrix.copy()
    for i in range(matrix.shape[0]):
        row = matrix[i]
        # Находим первый ненулевой элемент
        for val in row:
            if val != 0:
                if val < 0:
                    matrix[i] *= -1
                break  # выходим после первого ненулевого
    return matrix

def canonical_form_by_rows(matrix: np.ndarray) -> np.ndarray:
    """
    Приводит матрицу к каноническому виду через сортировку строк.
    
    matrix — исходная матрица (np.ndarray)
    Возвращает: новая матрица с отсортированными строками
    """

    # Сортируем строки 
    idx = np.lexsort(matrix.T[::-1])  # Инвертируем столбцы для нужного порядка
    return matrix[idx]

# Список матриц решетки grid
list_coords = []

for var in itertools.product(range(-1, 2), repeat=4): 
    # меньше 3 нельзя - тк при поиске центр региона отбрасываем незаконченные регионы
    #if var
    list_coords.append(var)
    
list_coords.remove((1, 0, 0, 0))
    
print(len(list_coords), list_coords[12])

result = []
for lst in list_coords:
    first_non_zero = next((x for x in lst if x != 0), None)
    if first_non_zero is not None and first_non_zero > 0:
        result.append(lst)

print(len(result))

#result = [np.random.rand(4) for _ in range(10)]  # список векторов
list_grids = []  # список уникальных базисов
c = 0  # счётчик хороших матриц
k = 0  # общий счётчик

# Общее количество комбинаций (для корректного tqdm)
n = len(result)
total_combinations = comb(n, 3)

print(f"Всего комбинаций: {total_combinations}")

# Основной цикл с прогресс-баром
for vectors in tqdm(combinations(result, 3), total=total_combinations, desc="Обработка комбинаций"):
    g = np.vstack(([1, 0, 0, 0], *vectors))  # Собираем векторы в матрицу 4x4
    k += 1
    
    if not np.isclose(np.linalg.det(g), 0):  # Если матрица невырожденная
        # Проверяем уникальность (без учёта транспонирования)
        #k += 1

        #grid_int = (g).astype(int)
        #basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
        #rows, cols = basis.nrows, basis.ncols
    
        #lll_grid = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

        lll_grid = LLL(g)
        
        lll_grid = normalize_rows(lll_grid) # делаем все первые элементы неотрицательными
        lll_grid = canonical_form_by_rows(lll_grid)

        if not any(np.allclose(lll_grid, m) for m in list_grids):
        #if not any(np.allclose(g.T, m) for m in list_grids):
            c += 1
            list_grids.append(lll_grid)

80 (-1, 0, 0, -1)
39
Всего комбинаций: 9139


Обработка комбинаций: 100%|██████████| 9139/9139 [00:01<00:00, 8286.34it/s] 


In [5]:
len(list_grids)

42

In [6]:
# перебор матриц решетки (float вариант)

import random

precision = 3  # точность округления

def generate_matrix():
    # Фиксируем первую строку
    first_row = [1.0, 0.0, 0.0, 0.0]
    
    while True:
        # Генерируем оставшиеся 3 строки
        matrix = [first_row]
        for _ in range(3):
            # Первый элемент — только от -1 до 1
            first = round(random.uniform(0, 1), precision)
            # Остальные — от -2 до 2
            rest = [round(random.uniform(-2, 2), precision) for _ in range(3)]
            row = [first] + rest
            matrix.append(row)
        
        mat = np.array(matrix, dtype=float)
        
        # Проверяем невырожденность
        if not np.isclose(np.linalg.det(mat), 0, atol=1e-8):
            return mat

In [7]:
# проверка что длина векторов больше 1

def vector_length(v):
    return np.linalg.norm(v)

def is_valid_matrix(matrix):
    for row in matrix:
        if vector_length(row) < 1:
            return False
    return True


In [8]:
list_grids = []  # список уникальных базисов
c = 0  # счётчик хороших матриц
#k = 0  # общий счётчик
scale = 10**precision  # Масштаб для округления
max_attempts = 100  # Максимальное количество попыток генерации матрицы

for i in range(max_attempts):
    matrix = generate_matrix()

    if not np.isclose(np.linalg.det(matrix), 0):  # Если матрица невырожденная
    #if is_valid_matrix(matrix):# проверяем что длина векторов больше 1

        # строим LLL базис
        #scale_mat = np.round(matrix * scale).astype(int)
        #grid_int = (scale_mat).astype(int)
        #basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
        #rows, cols = basis.nrows, basis.ncols
        
        # проверяем уникальность 
        #lll_grid = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)]) 
        #lll_grid = lll_grid * 1/scale # возвращаем float

        lll_grid = LLL(matrix)
        lll_grid = normalize_rows(lll_grid) # делаем все первые элементы неотрицательными
        lll_grid = canonical_form_by_rows(lll_grid) # сортируем строки

        #print(lll_grid, matrix, np.linalg.det(matrix))
        #print(66)
        
        # Проверяем уникальность (без учёта транспонирования)
        if not any(np.allclose(lll_grid, m) for m in list_grids):
            c += 1
            list_grids.append(lll_grid)

    else:
        # Пропускаем эту матрицу
        continue

len(list_grids), list_grids[0]

(100,
 array([[ 0.081, -0.04 ,  0.359,  0.743],
        [ 0.223, -1.11 ,  0.808, -0.377],
        [ 0.461,  0.199,  0.206, -0.231],
        [ 0.539, -0.199, -0.206,  0.231]]))

In [ ]:
for grid in list_grids:
    print(grid)
    vor4 = VoronoiPolyhedra(grid)
    vor4.build()

    det_dist, det_center, det_mat = find_optimal(range(49, 50), 2, grid, vor4, vor4.max_len)

    #print(det_dist, det_center, det_mat)

[[ 0.081 -0.04   0.359  0.743]
 [ 0.223 -1.11   0.808 -0.377]
 [ 0.461  0.199  0.206 -0.231]
 [ 0.539 -0.199 -0.206  0.231]]
суммарное расстояние = 100.57659911902618 индекс центрального региона = 151
Все точки используются в триангуляции.
Выпуклость триангуляции сохранена: True
                                                          
---------------------------------
det: 49
                                                          
▶ diag factors: 1 7 7    iters: 16807
                                                          
▶ diag factors: 1 1 49    iters: 117649
[ 99.87 % ]    iter: 117500[[ 0.217  0.076  1.212  2.054]
 [ 0.229 -1.81  -0.396  1.217]
 [ 0.316 -1.414  1.836 -0.485]
 [ 1.     0.     0.     0.   ]]
суммарное расстояние = 211.53454276330757 индекс центрального региона = 118
Все точки используются в триангуляции.
Выпуклость триангуляции сохранена: True
                                                          
---------------------------------
det: 49
               

Прочее

In [4]:
#grid[0][0] = 2.15

print(grid)

grid_int = (grid).astype(int)
basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
rows, cols = basis.nrows, basis.ncols
grid = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

vor4 = VoronoiPolyhedra(grid)
vor4.build()

det_dist, det_center, det_mat = find_optimal(range(49, 50), 2, grid, vor4, vor4.max_len)

print(det_dist, det_center, det_mat)

[[2. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 0. 1.]]
суммарное расстояние = 24.0 индекс центрального региона = 1223
Все точки используются в триангуляции.
Выпуклость триангуляции сохранена: True
                                                          
---------------------------------
det: 49
                                                          
▶ diag factors: 1 7 7    iters: 16807
 1.0801234497346432 [ 1 -2  0  3] [[1. 0. 0. 2.]
 [0. 1. 3. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0801234497346432 [ 0 -3  1  2] [[1. 0. 1. 4.]
 [0. 1. 3. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0801234497346432 [1 0 2 3] [[1. 0. 1. 2.]
 [0. 1. 5. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0801234497346432 [-1 -3  0  2] [[1. 0. 0. 4.]
 [0. 1. 5. 0.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0801234497346432 [ 1  0 -2  3] [[1. 0. 3. 0.]
 [0. 1. 0. 2.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0801234497346432 [ 0 -1 -3 -2] [[1. 0. 5. 6.]
 [0. 1. 0. 2.]
 [0. 0. 7. 0.]
 [0. 0. 0. 7.]]
 1.0801234497346432 [1 2 0 3] [[1

KeyboardInterrupt: 

In [7]:
vor4.polyhedrons[0].normal @ (vor4.polyhedrons[0].vertices[0]-vor4.polyhedrons[0].vertices[1])


np.float64(-0.7071067811865475)

In [10]:
vor4.vertices[vor4.list_edges]

array([[[ 0.5, -0.5,  0.5,  0.5],
        [ 0. ,  0. ,  1. ,  0. ]],

       [[ 0.5, -0.5,  0.5,  0.5],
        [ 0.5,  0.5,  0.5,  0.5]],

       [[ 0. ,  0. ,  1. ,  0. ],
        [ 0.5,  0.5,  0.5,  0.5]],

       [[ 0.5, -0.5,  0.5, -0.5],
        [ 0.5, -0.5,  0.5,  0.5]],

       [[ 1. ,  0. ,  0. ,  0. ],
        [ 0.5, -0.5,  0.5,  0.5]],

       [[ 0.5, -0.5,  0.5, -0.5],
        [ 1. ,  0. ,  0. ,  0. ]],

       [[ 0.5, -0.5,  0.5, -0.5],
        [ 0. ,  0. ,  1. ,  0. ]],

       [[ 0. ,  0. ,  1. ,  0. ],
        [ 0.5,  0.5,  0.5, -0.5]],

       [[ 0.5, -0.5,  0.5, -0.5],
        [ 0.5,  0.5,  0.5, -0.5]],

       [[ 1. ,  0. ,  0. ,  0. ],
        [ 0.5,  0.5,  0.5,  0.5]],

       [[ 0.5,  0.5,  0.5,  0.5],
        [ 0.5,  0.5,  0.5, -0.5]],

       [[ 1. ,  0. ,  0. ,  0. ],
        [ 0.5,  0.5,  0.5, -0.5]],

       [[ 0.5, -0.5,  0.5,  0.5],
        [-0.5, -0.5,  0.5,  0.5]],

       [[ 0. ,  0. ,  0. ,  1. ],
        [-0.5, -0.5,  0.5,  0.5]],

       [[ 0.5, -0.5,

In [3]:
#grid[0][0] = 2.15

print(grid)

grid_int = (grid).astype(int)
basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
rows, cols = basis.nrows, basis.ncols
grid = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

vor4 = VoronoiPolyhedra(grid)
vor4.build()

det_dist, det_center, det_mat = find_optimal(range(50, 60), 2, grid, vor4, vor4.max_len)

print(det_dist, det_center, det_mat)

[[2. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 0. 1.]]
суммарное расстояние = 24.0 индекс центрального региона = 1223
Все точки используются в триангуляции.
Выпуклость триангуляции сохранена: True
                                                          
---------------------------------
det: 50
                                                          
▶ diag factors: 2 5 5    iters: 6250
                                                          
▶ diag factors: 1 2 25    iters: 62500
                                                          
▶ diag factors: 1 5 10    iters: 25000
                                                          
▶ diag factors: 1 1 50    iters: 125000
                                                          
---------------------------------
det: 51
                                                          
▶ diag factors: 1 3 17    iters: 44217
                                                          
▶ diag factors: 1 1 51    iters: 132651
       

In [20]:
for i in range(len(vor4.polyhedrons)):
    
    print(vor4.polyhedrons[i].normal @ (vor4.polyhedrons[i].vertices[2] - vor4.polyhedrons[i].vertices[1]))

3.451271471459426e-17
8.377804252152117e-17
-9.557164560954655e-17
-4.146063078686082e-17
-7.750967338353505e-17
4.146063078686082e-17
-3.92696825423323e-17
-8.324935577164881e-17
-3.925231146709437e-17
1.1818404405555348e-18
7.850462293418875e-17
-6.284538892792202e-17
1.5659234006266723e-17
7.041607054573827e-17
-9.706984142572384e-17
8.818600258941368e-19
-1.9626155733547187e-16
3.92696825423323e-17
-6.284538892792202e-17
-2.359307746082765e-17
7.110376214092937e-17
3.96425957463291e-17
-2.091192546139133e-16
-4.180609606002592e-17
-1.362687828560688e-16
-7.945783998345824e-17


In [22]:
x = 8
def test():
    x = 7
    def l():
        #nonlocal x
        global x
        x = 1
        #return x
    x = l()
    print(x)

test()
print(x)

None
1
